In [ ]:
import torch
import cv2
import os
from pathlib import Path
from tqdm import tqdm
from google.colab import drive

# === Mount Google Drive ===
drive.mount('/content/drive')

# === Load YOLO model ===
model = torch.hub.load('ultralytics/yolov5', 'yolov5l', pretrained=True)
model.eval()

# === Input and Output Paths ===
input_dir = '/content/drive/MyDrive/AISC/Furniture Unzipped Files'
output_dir = '/content/drive/MyDrive/AISC/Detected_Images'
Path(output_dir).mkdir(parents=True, exist_ok=True)

# === Map class names to class IDs (optional) ===
name_to_id = {name: idx for idx, name in enumerate(model.names)}

# === Process each class folder ===
for class_folder in os.listdir(input_dir):
    class_path = os.path.join(input_dir, class_folder)
    if not os.path.isdir(class_path):
        continue

    # Create output folder for this class
    save_class_folder = os.path.join(output_dir, class_folder)
    os.makedirs(save_class_folder, exist_ok=True)

    # Open one text file for this class
    class_txt_path = os.path.join(output_dir, f"{class_folder}.txt")
    class_txt_file = open(class_txt_path, "w")

    # Process images inside the folder
    for img_file in tqdm(os.listdir(class_path), desc=f"Processing {class_folder}"):
        if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        img_path = os.path.join(class_path, img_file)
        image = cv2.imread(img_path)
        if image is None:
            continue

        h, w, _ = image.shape  # Image dimensions

        # Run YOLO detection
        results = model(img_path)
        detections = results.xyxy[0]  # [x1, y1, x2, y2, conf, cls]

        for *box, conf, cls in detections:
            x1, y1, x2, y2 = map(float, box)
            label_id = int(cls)  # Class ID (numeric)

            # Draw bounding box manually
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            label_text = f"{model.names[label_id]} {conf:.2f}"
            cv2.putText(image, label_text, (int(x1), int(y1) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Convert to YOLO format
            x_center = ((x1 + x2) / 2) / w
            y_center = ((y1 + y2) / 2) / h
            box_width = (x2 - x1) / w
            box_height = (y2 - y1) / h

            confidence = float(conf)

            # Write detection info into the class-wide txt file
            class_txt_file.write(f"{label_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f} {confidence:.4f} {img_file}\n")

        # Save manually annotated image into the corresponding output folder
        output_img_path = os.path.join(save_class_folder, img_file)
        cv2.imwrite(output_img_path, image)

    # Close the label text file after finishing the folder
    class_txt_file.close()

print("✅ Detection, annotation saving, and YOLO-format label generation completed correctly.")


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-4-29 Python-3.11.12 torch-2.6.0+cu124 CPU



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Streaming output truncated to the last 5000 lines.
Processing tv:  16%|█▌        | 483/2982 [15:15<1:13:47,  1.77s/it]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Processing tv:  16%|█▌        | 484/2982 [15:17<1:16:09,  1.83s/it]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Processing tv:  16%|█▋        | 485/2982 [15:19<1:16:52,  1.85s/it]/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Processing tv:  16%|█▋        | 486/2982 [15:20<1:09:44,  1.68s/it]/root/

✅ Detection, annotation saving, and YOLO-format label generation completed correctly.
